In [1]:
from nltk.corpus import stopwords
from string import punctuation
import pickle
import os

In [2]:
home_dir=os.getcwd()+'/'
# home_dir='/Users/vivek/My Folder/Learning/imdb_sentiment_project/review_polarity/'

dict_sent_mapping = {
        0: 'Negative',
        1: 'Positive',
    }

In [3]:
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 

def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 

In [4]:
def get_vocab(vocab_filename = 'vocab.txt'):
    # vocab_filename = 'vocab.txt'
    vocab = load_doc(os.getcwd()+'/'+vocab_filename)
    vocab = vocab.split()
    vocab = set(vocab)
    return vocab



In [5]:
vocab=get_vocab()

In [6]:
# classify a review as negative (0) or positive (1)
def predict_sentiment(review, vocab, tokenizer, model):
	tokens = clean_doc(review)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	# convert to line
	line = ' '.join(tokens)
	# encode
	encoded = tokenizer.texts_to_matrix([line], mode='freq')
	# prediction
	yhat = model.predict(encoded, verbose=0)
	return round(yhat[0,0])
#Rounding won't work well with multi-class classifiers.

In [7]:
# classify a review as negative (0) or positive (1)
def predict_sentiment_with_probability(review, vocab, tokenizer, model):
	# clean
	tokens = clean_doc(review)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	# convert to line
	line = ' '.join(tokens)
	# encode
	encoded = tokenizer.texts_to_matrix([line], mode='freq')
	# prediction
	yproba = model.predict_proba(encoded, verbose=0)    
# 	print(yproba)
	#model.predict() and model.predict_proba() gives the same result. For classes, use model.predict_classes()
	yhat = model.predict_classes(encoded, verbose=0)
	sent=dict_sent_mapping[yhat[0][0]] #Using dictionary to reverse mapping of class to label
# 	print(sent,yproba)    
	return (sent,yproba)  

In [ ]:
def get_tokenizer_and_model(tok_name='tokenizer.pickle', model_name=''):
    #
    # loading tokenizer
    with open(tok_name, 'rb') as handle:
        tokenizer = pickle.load(handle)
    #
    # load the model from disk
filename='finalized_model.sav'
model=loaded_model = pickle.load(open(filename, 'rb'))
    
    return tokenizer, model


In [8]:


    


#Write function to load tokenizer and model later
   

Using TensorFlow backend.


In [9]:
# test positive text
text = 'Best movie ever!'
print(predict_sentiment(text, vocab, tokenizer, model))
# test negative text
text = 'This is a bad movie.'
print(predict_sentiment(text, vocab, tokenizer, model))

1.0
0.0


In [10]:
# test positive text
text = '''The legend of Bhagat singh is one of the best movie ever! 
It is a cinematic masterpiece like none other.'''  # test positive text
print(predict_sentiment_with_probability(text, vocab, tokenizer, model))
# test negative text
text = '''Panipat is an awful movie. I don't think that such a bad movie has been made since Farah Khan's Joker. '''
print(predict_sentiment_with_probability(text, vocab, tokenizer, model))

('Positive', array([[0.98006564]], dtype=float32))
('Negative', array([[1.5453779e-07]], dtype=float32))


In [11]:
text = 'good'
print(predict_sentiment(text, vocab, tokenizer, model))
# test negative text
text = 'bad'
print(predict_sentiment(text, vocab, tokenizer, model))

1.0
0.0


In [ ]:
#Things that you can improve on:
    #1. Put all helper functions in a different file and import it
    #2. Figure out how to connect this to a flask api and provide an end point.
    #3. Tokenizer and model can be put in a separate function.
    